In [46]:
from ipyleaflet import Map, basemaps, DrawControl
map = Map(
    center = ( 40, -2 ), 
    zoom = 8,
    basemap = basemaps.OpenTopoMap
)

draw_control = DrawControl()

draw_control.circle = {
    "shapeOptions": {
        "fillColor": "#000000",
        "color": "#efed69",
        "fillOpacity": 1.0
    }
}

coordinates = []

def store_point( self, action, geo_json):
    global coordinates
    print(geo_json)
    coordinates = geo_json['geometry']['coordinates']
    
draw_control.on_draw(store_point)

map.add_control(draw_control)

map

Map(center=[40, -2], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

{'type': 'Feature', 'properties': {'style': {'stroke': True, 'color': '#efed69', 'weight': 4, 'opacity': 0.5, 'fill': True, 'fillColor': '#000000', 'fillOpacity': 1, 'clickable': True, 'radius': 1403.208532678574}}, 'geometry': {'type': 'Point', 'coordinates': [-3.062659, 40.03154]}}
{'type': 'Feature', 'properties': {'style': {'stroke': True, 'color': '#efed69', 'weight': 4, 'opacity': 0.5, 'fill': True, 'fillColor': '#000000', 'fillOpacity': 1, 'clickable': True, 'radius': 2703.838847236119}}, 'geometry': {'type': 'Point', 'coordinates': [-4.293383, 40.626183]}}


In [1]:
from pyproj import Proj, transform

coordinates = [-4.293383, 40.626183]
print(coordinates)
inProj = Proj(init='epsg:4326')
outProj = Proj(init='epsg:25830')
x, y = transform(inProj, outProj, coordinates[0], coordinates[1])

width = 2000

minx = x - width / 2
maxx = x + width / 2
miny = y - width / 2
maxy = y + width / 2

inProj = Proj(init='epsg:25830')
outProj = Proj(init='epsg:4258')

a, b = transform(inProj, outProj, minx, miny)
c, d = transform(inProj, outProj, maxx, maxy)

bbox = (a, b, c, d)
print(bbox)

[-4.293383, 40.626183]
(-4.305028459134179, 40.61704350257892, -4.281734366598346, 40.63532135685948)


In [65]:
from owslib.wcs import WebCoverageService
wcs = WebCoverageService('https://servicios.idee.es/wcs-inspire/mdt',version='1.0.0')

crs = wcs['Elevacion4258_5'].boundingboxes[0]['nativeSrs']

# content = wcs.getCoverage(identifier='Elevacion4258_5', bbox=bbox, width=400, height=400, crs=crs, format='image/png')
# out = open('prueba.png', 'wb')
content = wcs.getCoverage(identifier='Elevacion4258_5', bbox=bbox, width=400, height=400, crs=crs, format='application/asc')
# print(content.read().splitlines()[405].split()[399])

out = open('pr.asc', 'wb')
out.write(content.read())
out.close

<function BufferedWriter.close>

In [3]:

from owslib.wms import WebMapService
wms = WebMapService('https://www.ign.es/wms-inspire/mapa-raster', version='1.1.1')
list(wms.contents)
wms.getOperationByName('GetMap').formatOptions
img = wms.getmap( layers=['mtn_rasterizado'],
                srs='EPSG:25830',
                bbox=(minx, miny, maxx, maxy),
                format='image/png',
                size=(1024,1024)
                )
out = open('orto.png', 'wb')
out.write(img.read())
out.close()

In [4]:
from owslib.wms import WebMapService
wms = WebMapService('https://www.ign.es/wms-inspire/pnoa-ma', version='1.1.1')
list(wms.contents)
wms.getOperationByName('GetMap').formatOptions
img = wms.getmap( layers=['OI.OrthoimageCoverage'],
                srs='EPSG:25830',
                bbox=(minx, miny, maxx, maxy),
                format='image/png',
                size=(1024,1024)
                )
out = open('pnoa.png', 'wb')
out.write(img.read())
out.close()